# Semantic Kernel na Ujumuishaji wa OpenBnB MCP Server

Notibuku hii inaonyesha jinsi ya kutumia Semantic Kernel na OpenBnB MCP server halisi kutafuta malazi ya kweli ya Airbnb kwa kutumia MCPStdioPlugin. Kwa Upatikanaji wa LLM, inatumia Azure AI Foundry. Ili kusanidi vigezo vya mazingira yako, unaweza kufuata [Somo la Usanidi](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Kuunda Muunganisho wa Plugin ya MCP

Tutaunganishwa na [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) tukitumia MCPStdioPlugin. Seva hii inatoa uwezo wa kutafuta huduma za Airbnb kupitia kifurushi cha @openbnb/mcp-server-airbnb.


## Kuunda Mteja

Katika mfano huu, tutatumia Azure AI Foundry kwa ufikiaji wetu wa LLM. Hakikisha kwamba vigezo vya mazingira yako vimewekwa ipasavyo.


## Usanidi wa Mazingira

Sanidi mipangilio ya Azure OpenAI. Hakikisha una vigezo vya mazingira vifuatavyo vimewekwa:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Kuelewa Muunganisho wa OpenBnB MCP

Notebook hii inaunganisha na **seva halisi ya OpenBnB MCP** inayotoa uwezo wa kutafuta mali za Airbnb.

### Jinsi inavyofanya kazi:

1. **MCPStdioPlugin**: Inatumia mawasiliano ya pembejeo/pato la kawaida na seva ya MCP
2. **Kifurushi Halisi cha NPM**: Inapakua na kuendesha `@openbnb/mcp-server-airbnb` kupitia npx
3. **Takwimu za Moja kwa Moja**: Inarudisha data halisi ya mali za Airbnb kutoka kwa API zao
4. **Ugunduzi wa Kazi**: Wakala hugundua kiotomatiki kazi zinazopatikana kutoka kwa seva ya MCP

### Kazi Zinazopatikana:

Seva ya OpenBnB MCP kwa kawaida hutoa:
- **search_listings** - Tafuta mali za Airbnb kwa eneo na vigezo
- **get_listing_details** - Pata maelezo ya kina kuhusu mali maalum
- **check_availability** - Angalia upatikanaji kwa tarehe maalum
- **get_reviews** - Pata maoni kuhusu mali
- **get_host_info** - Pata maelezo kuhusu wenyeji wa mali

### Mahitaji:

- **Node.js** imewekwa kwenye mfumo wako
- **Muunganisho wa Intaneti** kupakua kifurushi cha seva ya MCP
- **NPX** kinapatikana (kinakuja na Node.js)

### Kupima Muunganisho:

Unaweza kupima seva ya MCP kwa mikono kwa kuendesha:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Hii itapakua na kuanzisha seva ya OpenBnB MCP, ambayo Semantic Kernel kisha inaunganisha nayo kwa data halisi ya Airbnb.


## Kuendesha Wakala na OpenBnB MCP Server

Sasa tutaendesha Wakala wa AI anayejumuika na OpenBnB MCP server kutafuta malazi halisi ya Airbnb huko Stockholm kwa watu wazima 2 na mtoto 1. Unaweza kubadilisha orodha ya `user_inputs` ili kurekebisha vigezo vya utafutaji.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Muhtasari
Hongera! Umefanikiwa kujenga wakala wa AI anayejumuika na utafutaji wa malazi halisi kwa kutumia Model Context Protocol (MCP):

## Teknolojia Zilizotumika:
- Semantic Kernel - Kwa kujenga mawakala wenye akili kwa kutumia Azure OpenAI
- Azure AI Foundry - Kwa uwezo wa LLM na kukamilisha mazungumzo
- MCP (Model Context Protocol) - Kwa ujumuishaji wa zana uliosanifishwa
- OpenBnB MCP Server - Kwa utafutaji halisi wa Airbnb
- Node.js/NPX - Kwa kuendesha seva ya nje ya MCP

## Ulichojifunza:
- Ujumuishaji wa MCP: Kuunganisha mawakala wa Semantic Kernel na seva za nje za MCP
- Ufikiaji wa Data kwa Wakati Halisi: Kutafuta mali halisi za Airbnb kupitia API za moja kwa moja
- Mawasiliano ya Itifaki: Kutumia mawasiliano ya stdio kati ya wakala na seva ya MCP
- Ugunduzi wa Kazi: Kugundua kiotomatiki kazi zinazopatikana kutoka kwa seva za MCP
- Majibu ya Mfululizo: Kunasa na kuhifadhi kumbukumbu za miito ya kazi kwa wakati halisi
- Utoaji wa HTML: Kuweka majibu ya wakala kwa meza zenye mitindo na maonyesho yanayoshirikiana

## Hatua Zifuatazo:
- Jumuisha seva za ziada za MCP (hali ya hewa, safari za ndege, migahawa)
- Jenga mfumo wa mawakala wengi unaochanganya MCP na itifaki za A2A
- Unda seva za MCP maalum kwa vyanzo vyako vya data
- Tekeleza kumbukumbu ya mazungumzo ya kudumu kati ya vikao
- Peleka wakala kwenye Azure Functions na uratibu wa seva ya MCP
- Ongeza uthibitishaji wa mtumiaji na uwezo wa kuhifadhi nafasi



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, inashauriwa kutumia huduma ya tafsiri ya kitaalamu ya binadamu. Hatutawajibika kwa maelewano mabaya au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
